# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn import multioutput

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
#Load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('message_n_categories', engine)

X = df['message']
y = df[df.columns[4:]]

In [6]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
   # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # remove stop words
    stop_words = stopwords.words("english")
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

In [16]:
# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred]), target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.77      0.45      0.57      1111
               request       0.00      0.00      0.00        33
                 offer       0.73      0.61      0.66      2670
           aid_related       0.67      0.12      0.21       535
          medical_help       0.75      0.10      0.17       344
      medical_products       0.60      0.04      0.07       159
     search_and_rescue       0.00      0.00      0.00       116
              security       0.56      0.12      0.20       200
              military       0.00      0.00      0.00         0
           child_alone       0.82      0.34      0.48       418
                 water       0.83      0.46      0.59       745
                  food       0.81      0.34      0.48       581
               shelter       0.69      0.11      0.19        98
              clothing       0.62      0.04      0.07       133
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [19]:
for idx, col in enumerate(y.columns):
    print(col, classification_report(y_test.iloc[:,idx], y_pred[:,idx]))

related              precision    recall  f1-score   support

          0       0.62      0.47      0.53      1563
          1       0.84      0.91      0.87      4944
          2       0.35      0.15      0.21        47

avg / total       0.78      0.80      0.79      6554

request              precision    recall  f1-score   support

          0       0.90      0.97      0.93      5443
          1       0.77      0.45      0.57      1111

avg / total       0.87      0.88      0.87      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.76      0.84      0.80      3884
          1       0.73      0.61      0.66      2670

avg / total       0.75      0.75      0.74      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
%%time
#grid search

parameters = {'clf__estimator__n_estimators': (5 ,10)}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=3)

CPU times: user 80 µs, sys: 1 µs, total: 81 µs
Wall time: 86.5 µs


In [24]:
%%time
cv.fit(X_train, y_train)
print('Best Parameters:', cv.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=5 ..................................
[CV]  clf__estimator__n_estimators=5, score=0.1956057369545316, total=  31.6s
[CV] clf__estimator__n_estimators=5 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   42.1s remaining:    0.0s


[CV]  clf__estimator__n_estimators=5, score=0.19499542264266098, total=  32.1s
[CV] clf__estimator__n_estimators=5 ..................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__n_estimators=5, score=0.20430271589868781, total=  32.0s
[CV] clf__estimator__n_estimators=10 .................................
[CV]  clf__estimator__n_estimators=10, score=0.232834909978639, total=  49.8s
[CV] clf__estimator__n_estimators=10 .................................
[CV]  clf__estimator__n_estimators=10, score=0.23466585291425085, total=  50.4s
[CV] clf__estimator__n_estimators=10 .................................
[CV]  clf__estimator__n_estimators=10, score=0.2494659749771132, total=  50.4s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  5.2min finished


Best Parameters: {'clf__estimator__n_estimators': 10}
CPU times: user 6min 30s, sys: 5.29 s, total: 6min 35s
Wall time: 6min 37s


In [25]:
y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
for idx, col in enumerate(y.columns):
    print(col, classification_report(y_test.iloc[:,idx], y_pred_cv[:,idx]))

related              precision    recall  f1-score   support

          0       0.62      0.47      0.53      1563
          1       0.84      0.91      0.87      4944
          2       0.35      0.15      0.21        47

avg / total       0.78      0.80      0.79      6554

request              precision    recall  f1-score   support

          0       0.90      0.97      0.93      5443
          1       0.77      0.45      0.57      1111

avg / total       0.87      0.88      0.87      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.76      0.84      0.80      3884
          1       0.73      0.61      0.66      2670

avg / total       0.75      0.75      0.74      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred_cv]), target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.77      0.45      0.57      1111
               request       0.00      0.00      0.00        33
                 offer       0.73      0.61      0.66      2670
           aid_related       0.67      0.12      0.21       535
          medical_help       0.75      0.10      0.17       344
      medical_products       0.60      0.04      0.07       159
     search_and_rescue       0.00      0.00      0.00       116
              security       0.56      0.12      0.20       200
              military       0.00      0.00      0.00         0
           child_alone       0.82      0.34      0.48       418
                 water       0.83      0.46      0.59       745
                  food       0.81      0.34      0.48       581
               shelter       0.69      0.11      0.19        98
              clothing       0.62      0.04      0.07       133
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
# Train the adjusted pipeline.
pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [30]:
%%time
parameters_ada = {  
              'clf__estimator__n_estimators': [100, 200],
              'clf__estimator__learning_rate': [0.1,0.3] }

cv_ada = GridSearchCV(pipeline_ada, param_grid=parameters_ada)
cv_ada.fit(X_train, y_train)

print('Best Parameters:', cv_ada.best_params_)

Best Parameters: {'clf__estimator__learning_rate': 0.3, 'clf__estimator__n_estimators': 200}
CPU times: user 54min 48s, sys: 9.79 s, total: 54min 57s
Wall time: 55min 5s


In [31]:
y_pred_ada = cv_ada.predict(X_test)

In [32]:
for idx, col in enumerate(y.columns):
    print(col, classification_report(y_test.iloc[:,idx], y_pred_ada[:,idx]))

related              precision    recall  f1-score   support

          0       0.67      0.10      0.18      1563
          1       0.77      0.98      0.86      4944
          2       0.33      0.02      0.04        47

avg / total       0.74      0.77      0.70      6554

request              precision    recall  f1-score   support

          0       0.90      0.98      0.94      5443
          1       0.83      0.48      0.61      1111

avg / total       0.89      0.90      0.88      6554

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.76      0.88      0.82      3884
          1       0.78      0.59      0.67      2670

avg / total       0.77      0.76      0.76      6554

medical_help              precision    recall  f1-sco

### 9. Export your model as a pickle file

In [80]:
import pickle

In [81]:
pickle.dump(pipeline_ada, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.